In [ ]:
library(tidyverse)
library(repr)
library(digest)
library(infer)
library(gridExtra)

Select neccesary data

In [ ]:
history_data <- read_csv("data/history.csv")
statement_data <- read_csv("data/statement.csv") %>%
    filter(`Account Type` == "Chequing") %>%
    rbind(history_data)%>%
    select("Transaction Date", "Description 2", "CAD$")%>%
    rename("Transaction"="CAD$")
head(statement_data, n=10)

Categorize transactions

In [ ]:
lookup_data <- read_csv("data/lookup_table.csv")
head(lookup_data)

Merge transaction type with remaining data and remove vendor name

In [ ]:
merged_data <- merge(statement_data, lookup_data)%>%
    select(-"Description 2")
head(merged_data, n=10)

Separate Transaction date into multiple columns

In [ ]:
date_no_delimiter <- separate(merged_data,
                             col = "Transaction Date",
                             into = c("Month", "Day", "Year"),
                             sep = "/")
date_sorted <- arrange(date_no_delimiter, Year, Month, Day)
head(date_sorted)

Group by month

In [ ]:
monthly_summary <- aggregate(date_no_delimiter$Transaction, by=list(Type=date_no_delimiter$type,
                                                                Month=date_no_delimiter$Month,
                                                                Year=date_no_delimiter$Year),
                            FUN=sum)
monthly_summary

Visualizations

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 15)
usage_chart <- ggplot(monthly_summary, aes(x=Month, y=x, fill=Type)) +
    geom_bar(stat = "identity", position = position_stack(reverse = TRUE)) + 
    labs(x="Month", y="Dollars Spent", fill="Transaction Type")+
    scale_y_reverse()+ 
    scale_fill_brewer(palette='Set3')+
    theme(text = element_text(size = 20))
usage_chart